In [1]:
import pandas as pd

In [50]:
df = pd.read_csv('./data/geneFamily_counts.csv')
df = df.rename(columns={'Unnamed: 0' : 'geneFamily'}).set_index('geneFamily')

In [52]:
df.head()

,Tetrapisispora blattae,Tetrapisispora phaffii,Naumovozyma castellii,Naumovozyma dairenensis,Kluyveromyces lactis,Eremothecium gossypii,Vanderwaltozyma polyspora,Lachancea thermotolerans,Kazachstania africana,Eremothecium cymbalariae,...,Serpula lacrymans,Cladophialophora carrionii,Schizosaccharomyces cryophilus,Metarhizium acridum,Rhizopus delemar,Exserohilum turcicum,Eutypa lata,Heterobasidion irregulare,Sclerotinia sclerotiorum,Cordyceps militaris
geneFamily,,,,,,,,,,,,,,,,,,,,,
3NTVE,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3NTVF,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
3NTVG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
3NTVH,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3NTVI,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
